In [37]:
import requests
import json
import pandas as pd
import folium

def tsunami_data():
    url = 'http://apis.data.go.kr/1741000/TsunamiShelter3/getTsunamiShelter1List'
    params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==', 'pageNo' : '1', 'numOfRows' : '1000', 'type' : 'json' }

    response = requests.get(url, params=params)
    json_ob = json.loads(response.content)
    body = json_ob['TsunamiShelter'][1]['row']
    body = pd.json_normalize(body)
    return body

In [53]:
sido_list = list(body['sido_name'].unique())
sido_list

['강원도', '경상북도', '부산광역시', '울산광역시', '전라남도']

In [45]:
def shelter_map(data):
        m = folium.Map(
        location=[data['lat'].mean(), data['lon'].mean()],
        zoom_start= 7
        )
        for idx in data.index:
                text = data.loc[idx, 'shel_nm'] + '<br>상세주소 :' + data.loc[idx, 'address'] + '<br>수용 가능 인원수 :' + str(data.loc[idx, 'shel_av'])  + '<br>해변으로부터 거리 :' + str(data.loc[idx, 'lenth']) + 'M' + '<br>해발 높이 :' + str(data.loc[idx, 'height']) + '<br>해변으로부터 거리 :' + str(data.loc[idx, 'lenth']) + '<br>내진적용여부 :' + data.loc[idx, 'seismic'] 
                folium.Marker([data.loc[idx, 'lat'], data.loc[idx, 'lon']], icon = folium.Icon(color="red"), tooltip = text).add_to(m)
        return m

In [54]:

def flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}{DocumentType}'
        response = requests.get(url)
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['content']
        body = pd.json_normalize(body)
        return body
    

def flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}/10M{DocumentType}'
        response = requests.get(url)
        contents = response.text
        # pp.pprint(response.content)
        json_ob = json.loads(contents)
        body = json_ob['content']
        # body = json_ob['response']['body']['item']
        body = pd.json_normalize(body)
        return body

def rainfall_api():
    serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
    HydroType = 'rainfall'
    DataType = 'list'
    DocumentType = '.json'
    water_level_live = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
    DataType = 'info'
    water_level = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
    water_level['시도명'] = water_level['addr'].str.split(' ').str[0]
    water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
    water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
    rainfall_df = pd.merge(water_level, water_level_live, on = 'rfobscd', how = 'inner')
    return rainfall_df

In [59]:
import pandas as pd
df = pd.read_excel("C:/disaster_project/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20221027).xlsx")
df = df.groupby(['1단계','2단계'])[['격자 X', '격자 Y']].agg(pd.Series.mode).reset_index()
# df = df[df['1단계'] != '이어도'].reset_index(drop = True)
# df.columns = ['cd_nm', 'lon', 'lat']
# df.to_csv('korea_cd_latlon.csv', index = False, encoding = 'utf-8-sig')
df['격자 X'] = df['격자 X'].apply(lambda x : x[0] if str(type(x)) == "<class 'numpy.ndarray'>" else x)
df['격자 Y'] = df['격자 Y'].apply(lambda x : x[0] if str(type(x)) == "<class 'numpy.ndarray'>" else x)
df.to_csv('korea_weatherlocation_xy.csv', encoding = 'utf-8-sig', index = False)

In [60]:
df

,1단계,2단계,격자 X,격자 Y
0,강원도,강릉시,92,132
1,강원도,고성군,85,145
2,강원도,동해시,97,127
3,강원도,삼척시,98,125
4,강원도,속초시,87,141
...,...,...,...,...
245,충청북도,청주시상당구,69,106
246,충청북도,청주시서원구,69,106
247,충청북도,청주시청원구,69,107
248,충청북도,청주시흥덕구,68,106


In [ ]:
import requests

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
params ={'serviceKey' : '서비스키', 'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'JSON', 'base_date' : '20210628', 'base_time' : '0630', 'nx' : '55', 'ny' : '127' }

response = requests.get(url, params=params)
print(response.content)

In [38]:
from datetime import datetime

base_date = datetime.now().strftime('%Y%m%d')
if int(datetime.now().strftime('%d')) > 30:
    base_time = datetime.now().strftime('%H30')
else :
    base_time = datetime.now().strftime('%H00')
    
base_date

'20221218'

In [52]:
type([1,2,3])

list

In [122]:
from datetime import datetime, timedelta
import requests
import json

# 시간 설정
base_datebf30 = datetime.now() - timedelta(minutes = 30)
base_date = base_datebf30.strftime('%Y%m%d')
if int(base_datebf30.strftime('%d')) > 30:
    base_time = base_datebf30.strftime('%H00')
else :
    base_time = base_datebf30.strftime('%H30')

print(base_date, base_time)
# api 호출준비(지역별 대기값)
korea_xy_df = pd.read_csv('C:/disaster_project/pages/using_data/korea_weatherlocation_xy.csv')

# 지역 선택
cd_nm_list = list(korea_xy_df['1단계'].unique())
cd_nm = '서울특별시'
sgg_nm_list = list(korea_xy_df[korea_xy_df['1단계'] == cd_nm]['2단계'].unique())
sgg_nm = '강서구'

# 격자 X, 격자 Y값 찾기
korea_xy_df = korea_xy_df[(korea_xy_df['1단계'] == cd_nm) & (korea_xy_df['2단계'] == sgg_nm)]
nx = korea_xy_df.iloc[0,2]
ny = korea_xy_df.iloc[0,3]

# api 호출
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==',
        'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'JSON', 'base_date' : base_date, 'base_time' : base_time, 'nx' : nx, 'ny' : ny }

# json csv 변환
response = requests.get(url, params=params)
contents = response.content
json_ob = json.loads(contents)
body = json_ob['response']['body']['items']['item']
body = pd.json_normalize(body)
body


20221219 0030


,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20221219,0030,LGT,20221219,0100,0,58,126
1,20221219,0030,LGT,20221219,0200,0,58,126
2,20221219,0030,LGT,20221219,0300,0,58,126
3,20221219,0030,LGT,20221219,0400,0,58,126
4,20221219,0030,LGT,20221219,0500,0,58,126
5,20221219,0030,LGT,20221219,0600,0,58,126
6,20221219,0030,PTY,20221219,0100,0,58,126
7,20221219,0030,PTY,20221219,0200,0,58,126
8,20221219,0030,PTY,20221219,0300,0,58,126
9,20221219,0030,PTY,20221219,0400,0,58,126


In [95]:
base_date
base_time

'0000'

In [99]:
# 시간 설정
base_date = datetime.now().strftime('%Y%m%d')
if int(datetime.now().strftime('%d')) > 30:
    base_time = datetime.now().strftime('%H30')
else :
    base_time = datetime.now().strftime('%H00')

In [101]:
datetime.now().strftime('%Y%m%d%H')

'2022121900'

In [106]:
from datetime import datetime, timedelta
base_date = datetime.now() - timedelta(minutes = 30)
base_date

datetime.datetime(2022, 12, 18, 23, 47, 52, 419856)

In [6]:

vin_df = pd.DataFrame()
pageNo = 1

pageNo = str(pageNo)
url = 'http://apis.data.go.kr/1741000/TemporaryHousingFacilityVictim3/getTemporaryHousingFacilityVictim1List'
params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==',
        'pageNo' : pageNo,
        'numOfRows' : '100',
        'type' : 'json' }

response = requests.get(url, params=params)
contents = response.text
json_ob = json.loads(contents)
body = json_ob['TemporaryHousingFacilityVictim'][1]['row']
body = pd.json_normalize(body)
vin_df = pd.concat([vin_df,body], axis = 0)
pageNo = int(pageNo) + 1
        
columns = ['ctprvn_nm', 'sgg_nm', 'acmdfclty_se_nm', 'vt_acmdfclty_nm', 'dtl_adres', 'fclty_ar', 'vt_acmd_psbl_nmpr', 'mngps_nm', 'mngps_telno', 'xcord', 'ycord']
vin_df2 = vin_df[columns]
vin_df2.columns = ['시도명', '시군구명', '시설구분', '시설명', '상세주소','시설면적', '주거능력','관리부서','지자체담당자연락처', '경도', '위도']
    
#     vin_df2.reset_index(drop = True).to_csv(os.path.join(filePath, 'pages','using_data', 'temporary_house.csv'), index = False, encoding = 'utf-8-sig')

<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>
		<returnReasonCode>30</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>
<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>
		<returnReasonCode>30</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>
<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>
		<returnReasonCode>30</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>
<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>
		<returnReasonCode>30</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>
<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<re

KeyError: "None of [Index(['ctprvn_nm', 'sgg_nm', 'acmdfclty_se_nm', 'vt_acmdfclty_nm',\n       'dtl_adres', 'fclty_ar', 'vt_acmd_psbl_nmpr', 'mngps_nm', 'mngps_telno',\n       'xcord', 'ycord'],\n      dtype='object')] are in the [columns]"

In [10]:
url = 'http://223.130.129.189:9191/getTsunamiShelter1List/numOfRows=100&pageNo=1&type=json'
response = requests.get(url)
contents = response.text

In [24]:
def TsunamiShelter():
    pageNo = 1
    numOfRows = 1000
    type = 'json'
    url = f'http://223.130.129.189:9191/getTsunamiShelter1List/numOfRows={numOfRows}&pageNo={pageNo}&type={type}'
    response = requests.get(url)
    json_ob = json.loads(response.content)
    body = json_ob['TsunamiShelter'][1]['row']
    body = pd.json_normalize(body)
    return body

TsunamiShelter()

,id,sido_name,sigungu_name,remarks,shel_nm,address,lon,lat,shel_av,lenth,shel_div_type,seismic,height
0,606,경상북도,울진군,금음지구3,구 7번국도,경상북도 울진군 후포면 금음리 385,129.436249,36.673766,500,300,도로변,,40
1,612,경상북도,울진군,삼율지구1,후포초등학교,경상북도 울진군 후포면 삼율5길 8,129.441419,36.681171,9000,350,학교운동장,,18
2,605,경상북도,울진군,금음지구2,국도옆산마루측구,경상북도 울진군 후포면 금음리 259,129.431039,36.660888,300,80,공터,,25
3,610,경상북도,울진군,금음지구2,반소골길,경상북도 울진군 후포면 금음리 산259,129.398723,36.665528,300,100,공터,,20
4,611,경상북도,울진군,금음지구1,어업무선국송신소,경상북도 울진군 후포면 동해대로 307-18,129.434671,36.669545,300,100,공터,,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,646,경상북도,고령군,선택안함,성산초등학교 운동장,경상북도 고령군 성산면 성산로 902,128.360870,35.748595,9975,0,,,0
646,647,전라남도,화순군,선택안함,도곡초등학교 운동장,전라남도 화순군 도곡면 천암1길 71,126.908600,35.013100,5091,0,,,0
647,648,경상북도,고령군,선택안함,성산중학교 운동장,경상북도 고령군 성산면 운성로 877,128.362581,35.750874,2525,0,,,0
648,649,전라남도,화순군,선택안함,도암중학교 운동장,전라남도 화순군 도암면 권동길 4-6,126.895500,34.937800,4848,0,,,0


In [ ]:
import bs4

# 공공데이터 조회
url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire'
params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==', 'STAGE1' : cd_nm, 'STAGE2' : sgg_nm, 'pageNo' : '1', 'numOfRows' : '1000' }


url = f'http://223.130.129.189:9191/getEgytBassInfoInqire/STAGE1={cd_nm}&STAGE2={sgg_nm}'

response = requests.get(url, params=params)
content = response.text

### xml을 DataFrame으로 변환하기 ###
#bs4 사용하여 item 태그 분리

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

# xml값 DataFrame으로 만들기
emergency_hospital_df = pd.DataFrame(row_list, columns=name_list)

# 데이터 가공
emergency_hospital_df = emergency_hospital_df[['hvidate','hvec','hvoc','hvgc','hvamyn','dutyName','dutyTel3']]
emergency_hospital_df.columns = ['정보 업데이트 일시','응급실 가용현황', '수술실 가용현황', '입원실 가용현황', '구급차 가용여부', '기관명', '연락처']
